# Vanilla OFA Experiments

These are the experiments to test the performance of the vanilla OFA implementation. If you are running an experiment to test the validity of a particular design space, please make sure to load the approrpiate OFA network before doing so. The cells corresponding to each design space have been marked with "DESIGN SPACE: <NAME>" at the top of the cell.
     
We do not recommend running any of the experiments as they may take a while (we average over 10 runs). Please run the cell under the DEMO section to see bottom-up in action. Before running it, make sure you run all cells in the PREP section

# PREP

In [6]:
print('Installing PyTorch...')
! pip install torch 1>/dev/null
print('Installing torchvision...')
! pip install torchvision 1>/dev/null
print('Installing numpy...')
! pip install numpy 1>/dev/null
# thop is a package for FLOPs computing.
print('Installing thop (FLOPs counter) ...')
! pip install thop 1>/dev/null
# ofa is a package containing training code, pretrained specialized models and inference code for the once-for-all networks.
print('Installing OFA...')
! pip install ofa 1>/dev/null
# tqdm is a package for displaying a progress bar.
print('Installing tqdm (progress bar) ...')
! pip install tqdm 1>/dev/null
print('Installing matplotlib...')
! pip install matplotlib 1>/dev/null
print('All required packages have been successfully installed!')

Installing PyTorch...
Installing torchvision...
Installing numpy...
Installing thop (FLOPs counter) ...
Installing OFA...
Installing tqdm (progress bar) ...
Installing matplotlib...
All required packages have been successfully installed!


Then, we can import the packages used in this tutorial:

In [12]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import numpy as np
import time
import random
import math
import copy
from matplotlib import pyplot as plt

from ofa.model_zoo import ofa_net
from ofa.utils import download_url

# from ofa.tutorial.accuracy_predictor import AccuracyPredictor
# from ofa.tutorial.flops_table import FLOPsTable
# from ofa.tutorial.latency_table import LatencyTable
# from ofa.tutorial.evolution_finder import EvolutionFinder
# from ofa.tutorial.imagenet_eval_helper import evaluate_ofa_subnet, evaluate_ofa_specialized
import ofa
from ofa.tutorial import AccuracyPredictor, FLOPsTable, LatencyTable, EvolutionFinder
from ofa.tutorial import evaluate_ofa_subnet, evaluate_ofa_specialized

# set random seed
random_seed = 1
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
print('Successfully imported all packages and configured random seed to %d!'%random_seed)

Successfully imported all packages and configured random seed to 1!


Now it's time to determine which device to use for neural network inference in the rest of this tutorial. If your machine is equipped with GPU(s), we will use the GPU by default. Otherwise, we will use the CPU.

In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda_available = torch.cuda.is_available()
if cuda_available:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.cuda.manual_seed(random_seed)
    print('Using GPU.')
else:
    print('Using CPU.')

Using CPU.


Good! Now you have successfully configured the environment! It's time to import the **OFA network** for the following experiments.
The OFA network used in this tutorial is built upon MobileNetV3 with width multiplier 1.2, supporting elastic depth (2, 3, 4) per stage, elastic expand ratio (3, 4, 6), and elastic kernel size (3, 5 7) per block.

In [3]:
ofa_network = ofa_net('ofa_mbv3_d234_e346_k357_w1.2', pretrained=True)
print('The OFA Network is ready.')

The OFA Network is ready.


Now, let's build the ImageNet dataset and the corresponding dataloader. Notice that **if you're using the CPU,
we will skip ImageNet evaluation by default** since it will be very slow.
If you are using the GPU, in case you don't have the full dataset,
we will download a subset of ImageNet which contains 2,000 images (~250M) for testing.
If you do have the full ImageNet dataset on your machine, just specify it in `imagenet_data_path` and the downloading script will be skipped.

In [4]:
if cuda_available:
    # path to the ImageNet dataset
    print("Please input the path to the ImageNet dataset.\n")
    imagenet_data_path = input()

    # if 'imagenet_data_path' is empty, download a subset of ImageNet containing 2000 images (~250M) for test
    if not os.path.isdir(imagenet_data_path):
        os.makedirs(imagenet_data_path, exist_ok=True)
        download_url('https://hanlab.mit.edu/files/OnceForAll/ofa_cvpr_tutorial/imagenet_1k.zip', model_dir='data')
        ! cd data && unzip imagenet_1k 1>/dev/null && cd ..
        ! cp -r data/imagenet_1k/* $imagenet_data_path
        ! rm -rf data
        print('%s is empty. Download a subset of ImageNet for test.' % imagenet_data_path)

    print('The ImageNet dataset files are ready.')
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.


Now you have configured the dataset. Let's build the dataloader for evaluation.
Again, this will be skipped if you are in a CPU environment.

In [5]:
if cuda_available:
    # The following function build the data transforms for test
    def build_val_transform(size):
        return transforms.Compose([
            transforms.Resize(int(math.ceil(size / 0.875))),
            transforms.CenterCrop(size),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])

    data_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(
            root=os.path.join(imagenet_data_path, 'val'),
            transform=build_val_transform(224)
        ),
        batch_size=250,  # test batch size
        shuffle=True,
        num_workers=16,  # number of workers for the data loader
        pin_memory=True,
        drop_last=False,
    )
    print('The ImageNet dataloader is ready.')
else:
    data_loader = None
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.


## 2. Using Pretrained Specialized OFA Sub-Networks
![](https://hanlab.mit.edu/files/OnceForAll/figures/select_subnets.png)
The specialized OFA sub-networks are "small" networks sampled from the "big" OFA network as is indicated in the figure above.
The OFA network supports over $10^{19}$ sub-networks simultaneously, so that the deployment cost for multiple scenarios can be saved by 16$\times$ to 1300$\times$ under 40 deployment scenarios.
Now, let's play with some of the sub-networks through the following interactive command line prompt (**Notice that for CPU users, this will be skipped**).
We recommend you to try a smaller sub-network (e.g., the sub-network for pixel1 with 20ms inference latency constraint) so that it takes less time to evaluate the model on ImageNet.

In [6]:
if cuda_available:
    net_id = evaluate_ofa_specialized(imagenet_data_path, data_loader)
    print('Finished evaluating the pretrained sub-network: %s!' % net_id)
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.


## 3 Efficient Deployment with OFA Networks

You have now successfully prepared the whole environment for the experiment!
In the next step, we will introduce **how to get efficient, specialized neural networks within minutes**
powered by the OFA network.

### 3.1 Latency-Constrained Efficient Deployment on Samsung Note10

The key components of very fast neural network deployment are **accuracy predictors** and **efficiency predictors**.
For the accuracy predictor, it predicts the Top-1 accuracy of a given sub-network on a **holdout validation set**
(different from the official 50K validation set) so that we do **NOT** need to run very costly inference on ImageNet
while searching for specialized models. Such an accuracy predictor is trained using an accuracy dataset built with the OFA network.

![](https://hanlab.mit.edu/files/OnceForAll/figures/predictor_based_search.png)

In [7]:
# accuracy predictor
accuracy_predictor = AccuracyPredictor(
    pretrained=True,
    device='cuda:0' if cuda_available else 'cpu'
)

print('The accuracy predictor is ready!')
print(accuracy_predictor.model)

The accuracy predictor is ready!
Sequential(
  (0): Linear(in_features=128, out_features=400, bias=True)
  (1): ReLU()
  (2): Linear(in_features=400, out_features=400, bias=True)
  (3): ReLU()
  (4): Linear(in_features=400, out_features=400, bias=True)
  (5): ReLU()
  (6): Linear(in_features=400, out_features=1, bias=True)
)


Now, we have the powerful **accuracy predictor**. We then introduce two types of **efficiency predictors**: the latency predictor and the FLOPs predictor. 

The intuition of having efficiency predictors, especially the latency predictor, is that measuring the latency of a sub-network on-the-fly is also costly, especially for mobile devices.
The latency predictor is designed to eliminate this cost.
Let's load a latency predictor we built beforehand for the Samsung Note10.

In [8]:
target_hardware = 'note10'
latency_table = LatencyTable(device=target_hardware)
print('The Latency lookup table on %s is ready!' % target_hardware)

Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/160_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/160_lookup_table.yaml
/Users/vidhur2k/anaconda3/lib/python3.7/site-packages/ofa/tutorial/latency_table.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.lut = yaml.load(fp)
Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/176_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/176_lookup_table.yaml


Built latency table for image size: 160.
Built latency table for image size: 176.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/192_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/192_lookup_table.yaml


Built latency table for image size: 192.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/208_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/208_lookup_table.yaml


Built latency table for image size: 208.


Downloading: "https://hanlab.mit.edu/files/OnceForAll/tutorial/latency_table@note10/224_lookup_table.yaml" to /Users/vidhur2k/.hancai/latency_tools/224_lookup_table.yaml


Built latency table for image size: 224.
The Latency lookup table on note10 is ready!


So far, we have defined both the accuracy predictor and the latency predictor. Now, let's experience **very fast model specialization** on Samsung Note10 with these two powerful predictors! 

**Notice**: The predicted accuracy is on a holdout validation set of 10K images, not the official 50K validation set.
But they are highly positive-correlated.

Great! You get your specialized neural network with **just a few seconds**!
You can go back to the last cell and modify the hyper-parameters to see how they affect the search time and the accuracy.

We also provided an interface below to draw a figure comparing your searched specialized network and other efficient neural networks such as MobileNetV3 and ProxylessNAS.

**Notice**: For ease of comparison, we recommend you to choose a latency constraint between 15ms and 33ms.

In [9]:
# evaluate the searched model on ImageNet
if cuda_available:
    top1s = []
    latency_list = []
    for result in result_lis:
        _, net_config, latency = result
        print('Evaluating the sub-network with latency = %.1f ms on %s' % (latency, target_hardware))
        top1 = evaluate_ofa_subnet(
            ofa_network,
            imagenet_data_path,
            net_config,
            data_loader,
            batch_size=250,
            device='cuda:0' if cuda_available else 'cpu')
        top1s.append(top1)
        latency_list.append(latency)

    plt.figure(figsize=(4,4))
    plt.plot(latency_list, top1s, 'x-', marker='*', color='darkred',  linewidth=2, markersize=8, label='OFA')
    plt.plot([26, 45], [74.6, 76.7], '--', marker='+', linewidth=2, markersize=8, label='ProxylessNAS')
    plt.plot([15.3, 22, 31], [73.3, 75.2, 76.6], '--', marker='>', linewidth=2, markersize=8, label='MobileNetV3')
    plt.xlabel('%s Latency (ms)' % target_hardware, size=12)
    plt.ylabel('ImageNet Top-1 Accuracy (%)', size=12)
    plt.legend(['OFA', 'ProxylessNAS', 'MobileNetV3'], loc='lower right')
    plt.grid(True)
    plt.show()
    print('Successfully draw the tradeoff curve!')
else:
    print('Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.')

Since GPU is not found in the environment, we skip all scripts related to ImageNet evaluation.


# Experiments

In [10]:
def run_vanilla_evolutionary_search(latency_constraints):
    st = time.time()
    for latency_constraint in latency_constraints:
        """ 
        Hyper-parameters for the evolutionary search process
        You can modify these hyper-parameters to see how they influence the final ImageNet accuracy of the search sub-net.
        """
#         latency_constraint = 25  # ms, suggested range [15, 33] ms
        P = 100  # The size of population in each generation
        N = 500  # How many generations of population to be searched
        r = 0.25  # The ratio of networks that are used as parents for next generation
        params = {
            'constraint_type': target_hardware, # Let's do FLOPs-constrained search
            'efficiency_constraint': latency_constraint,
            'mutate_prob': 0.1, # The probability of mutation in evolutionary search
            'mutation_ratio': 0.5, # The ratio of networks that are generated through mutation in generation n >= 2.
            'efficiency_predictor': latency_table, # To use a predefined efficiency predictor.
            'accuracy_predictor': accuracy_predictor, # To use a predefined accuracy_predictor predictor.
            'population_size': P,
            'max_time_budget': N,
            'parent_ratio': r,
        }

        # build the evolution finder
        finder = EvolutionFinder(**params)

        # start searching
        result_lis = []
        best_valids, best_info = finder.run_evolution_search()
        result_lis.append(best_info)
        print('Found best architecture on %s with latency <= %.2f ms '
              'It achieves %.2f%s predicted accuracy with %.2f ms latency on %s.' %
              (target_hardware, latency_constraint, best_info[0] * 100, '%', best_info[-1], target_hardware))

        # visualize the architecture of the searched sub-net
        _, net_config, latency = best_info
        ofa_network.set_active_subnet(ks=net_config['ks'], d=net_config['d'], e=net_config['e'])
        print('Architecture of the searched sub-net:')
        print(ofa_network.module_str)
    ed = time.time()
    return ed-st

In [ ]:
ofa_network = ofa_net('ofa_mbv3_d234_e346_k357_w1.2', pretrained=True)
#ofa_network = ofa.model_zoo.ofa_net('ofa_resnet50', pretrained=True)
print('The OFA Network is ready.')

In [13]:
ofa_network = ofa_net('ofa_resnet50', pretrained=True)
#ofa_network = ofa.model_zoo.ofa_net('ofa_resnet50', pretrained=True)
print('The OFA Network is ready.')

Downloading: "https://hanlab.mit.edu/files/OnceForAll/ofa_nets/ofa_resnet50_d=0+1+2_e=0.2+0.25+0.35_w=0.65+0.8+1.0" to .torch/ofa_nets/ofa_resnet50_d=0+1+2_e=0.2+0.25+0.35_w=0.65+0.8+1.0


The OFA Network is ready.


In [15]:
ofa_network = ofa_net('ofa_proxyless_d234_e346_k357_w1.3', pretrained=True)
#ofa_network = ofa.model_zoo.ofa_net('ofa_resnet50', pretrained=True)
print('The OFA Network is ready.')

Downloading: "https://hanlab.mit.edu/files/OnceForAll/ofa_nets/ofa_proxyless_d234_e346_k357_w1.3" to .torch/ofa_nets/ofa_proxyless_d234_e346_k357_w1.3


The OFA Network is ready.


## Running Time

### MobileNetV3

In [15]:
times = {}
latency_constraints = (15, 20, 25, 30, 35)
for i in range(len(latency_constraints)):
    latency_constraint = latency_constraints[:i+1]
    times[latency_constraint] = run_vanilla_evolutionary_search(latency_constraint)

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:19<00:00, 25.70it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.62% predicted accuracy with 14.81 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E3.0, K3), Identity)
(SE(O48, E3.0, K5), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K5), Identity)
((O96, E3.0, K5), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E4.0, K5), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E3.0, K3), Identity)
(SE(O192, E4.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (15): 100%|██████████| 500/500 [00:20<00:00, 24.35it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.48% predicted accuracy with 14.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E3.0, K5), None)
(SE(O48, E4.0, K3), Identity)
((O96, E3.0, K5), None)
((O96, E4.0, K3), Identity)
(SE(O136, E3.0, K7), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E4.0, K5), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (20): 100%|██████████| 500/500 [00:22<00:00, 22.66it/s]


...

Found best architecture on note10 with latency <= 25.00 ms It achieves 81.62% predicted accuracy with 24.99 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E4.0, K5), Identity)
(SE(O48, E3.0, K3), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E6.0, K5), None)
((O96, E4.0, K7), Identity)
((O96, E3.0, K5), Identity)
(SE(O136, E6.0, K7), None)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E6.0, K7), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E6.0, K7), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (35):   1%|          | 3/500 [00:00<00:20, 24.31it/s]

Found best architecture on note10 with latency <= 30.00 ms It achieves 82.70% predicted accuracy with 29.91 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E4.0, K3), None)
((O32, E6.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O48, E4.0, K5), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E6.0, K5), None)
((O96, E4.0, K7), Identity)
((O96, E3.0, K3), Identity)
((O96, E3.0, K7), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E6.0, K3), Identity)
(SE(O136, E4.0, K7), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E3.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (35): 100%|██████████| 500/500 [00:22<00:00, 22.56it/s]

Found best architecture on note10 with latency <= 35.00 ms It achieves 83.20% predicted accuracy with 34.92 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O48, E6.0, K3), Identity)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K7), None)
((O96, E4.0, K3), Identity)
((O96, E4.0, K5), Identity)
((O96, E4.0, K3), Identity)
(SE(O136, E6.0, K5), None)
(SE(O136, E6.0, K5), Identity)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E4.0, K7), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E3.0, K7), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



In [16]:
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
times[latency_constraints] = run_vanilla_evolutionary_search(latency_constraints)
times

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:27<00:00, 18.16it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.44% predicted accuracy with 14.93 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E4.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E4.0, K5), Identity)
((O96, E6.0, K3), Identity)
(SE(O136, E4.0, K7), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E3.0, K3), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E3.0, K3), Identity)
(SE(O192, E4.0, K5), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (25):   0%|          | 0/500 [00:00<?, ?it/s]

Found best architecture on note10 with latency <= 20.00 ms It achieves 80.46% predicted accuracy with 19.97 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K5), None)
(SE(O48, E4.0, K5), Identity)
(SE(O48, E6.0, K3), Identity)
((O96, E4.0, K7), None)
((O96, E4.0, K5), Identity)
((O96, E3.0, K3), Identity)
((O96, E3.0, K7), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E3.0, K7), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (30):   0%|          | 0/500 [00:00<?, ?it/s]

...

Searching with note10 constraint (60):   1%|          | 4/500 [00:00<00:16, 30.37it/s]

Found best architecture on note10 with latency <= 55.00 ms It achieves 84.83% predicted accuracy with 54.20 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E4.0, K3), None)
((O32, E3.0, K7), Identity)
((O32, E6.0, K3), Identity)
(SE(O48, E6.0, K5), None)
(SE(O48, E6.0, K5), Identity)
(SE(O48, E3.0, K7), Identity)
(SE(O48, E6.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E3.0, K7), Identity)
((O96, E6.0, K3), Identity)
((O96, E6.0, K7), Identity)
(SE(O136, E4.0, K5), None)
(SE(O136, E4.0, K7), Identity)
(SE(O136, E6.0, K7), Identity)
(SE(O136, E6.0, K5), Identity)
(SE(O192, E6.0, K7), None)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E4.0, K5), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (60): 100%|██████████| 500/500 [00:20<00:00, 23.94it/s]

Found best architecture on note10 with latency <= 60.00 ms It achieves 84.92% predicted accuracy with 59.83 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E4.0, K3), None)
((O32, E6.0, K5), Identity)
((O32, E4.0, K3), Identity)
((O32, E3.0, K5), Identity)
(SE(O48, E6.0, K7), None)
(SE(O48, E6.0, K3), Identity)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E4.0, K7), Identity)
((O96, E4.0, K7), None)
((O96, E3.0, K5), Identity)
((O96, E4.0, K5), Identity)
((O96, E6.0, K5), Identity)
(SE(O136, E6.0, K5), None)
(SE(O136, E6.0, K7), Identity)
(SE(O136, E6.0, K5), Identity)
(SE(O136, E6.0, K7), Identity)
(SE(O192, E6.0, K5), None)
(SE(O192, E6.0, K7), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E6.0, K7), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



{(15,): 27.0648090839386,
 (15, 20): 50.516709089279175,
 (15, 20, 25): 68.79455637931824,
 (15, 20, 25, 30): 96.6699378490448,
 (15, 20, 25, 30, 35): 123.10927200317383,
 (15, 20, 25, 30, 35, 40, 45, 50, 55, 60): 247.85819578170776}

### ResNet50D

In [14]:
times = {}
latency_constraints = (15, 20, 25, 30, 35)
for i in range(len(latency_constraints)):
    latency_constraint = latency_constraints[:i+1]
    times[latency_constraint] = run_vanilla_evolutionary_search(latency_constraint)
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
times[latency_constraints] = run_vanilla_evolutionary_search(latency_constraints)
times

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:17<00:00, 27.84it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.37% predicted accuracy with 14.97 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpoo

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:21<00:00, 23.38it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.61% predicted accuracy with 14.89 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
(DyConv(O32, K3, S1), Identity)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->768->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->1536->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpo

Searching with note10 constraint (20): 100%|██████████| 500/500 [00:20<00:00, 24.88it/s]


...

Searching with note10 constraint (60):   1%|          | 3/500 [00:00<00:19, 25.39it/s]

Found best architecture on note10 with latency <= 55.00 ms It achieves 84.82% predicted accuracy with 54.00 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->1024->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->768->256_S1, Identity)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->3072->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->12288->2048_S2, avgpool_conv)
(3x3_BottleneckConv_i

Searching with note10 constraint (60): 100%|██████████| 500/500 [00:19<00:00, 25.32it/s]

Found best architecture on note10 with latency <= 60.00 ms It achieves 84.96% predicted accuracy with 58.48 ms latency on note10.
Architecture of the searched sub-net:
DyConv(O32, K3, S2)
DyConv(O64, K3, S1)
max_pooling(ks=3, stride=2)
(3x3_BottleneckConv_in->1024->256_S1, avgpool_conv)
(3x3_BottleneckConv_in->1536->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->1024->256_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S2, avgpool_conv)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->2048->512_S1, Identity)
(3x3_BottleneckConv_in->3072->512_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S2, avgpool_conv)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->4096->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->6144->1024_S1, Identity)
(3x3_BottleneckConv_in->8192->2048_S2, avgpool_conv)
(3x3_BottleneckConv_i

{(15,): 26.64504313468933,
 (15, 20): 51.21063709259033,
 (15, 20, 25): 69.84648180007935,
 (15, 20, 25, 30): 91.103444814682,
 (15, 20, 25, 30, 35): 118.18609690666199,
 (15, 20, 25, 30, 35, 40, 45, 50, 55, 60): 227.73199486732483}

### ProxylessNAS

In [16]:
times = {}
latency_constraints = (15, 20, 25, 30, 35)
for i in range(len(latency_constraints)):
    latency_constraint = latency_constraints[:i+1]
    times[latency_constraint] = run_vanilla_evolutionary_search(latency_constraint)
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)
times[latency_constraints] = run_vanilla_evolutionary_search(latency_constraints)
times

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:20<00:00, 24.76it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.41% predicted accuracy with 15.00 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E6.0, K3), None)
((O56, E6.0, K3), Identity)
((O104, E4.0, K5), None)
((O104, E3.0, K7), Identity)
((O128, E4.0, K3), None)
((O128, E4.0, K5), Identity)
((O128, E4.0, K3), Identity)
((O248, E4.0, K3), None)
((O248, E6.0, K3), Identity)
((O248, E4.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



Searching with note10 constraint (15): 100%|██████████| 500/500 [00:23<00:00, 20.92it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.57% predicted accuracy with 14.91 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E3.0, K3), Identity)
((O56, E3.0, K3), Identity)
((O104, E6.0, K5), None)
((O104, E4.0, K5), Identity)
((O104, E3.0, K5), Identity)
((O128, E4.0, K7), None)
((O128, E4.0, K3), Identity)
((O128, E4.0, K5), Identity)
((O248, E4.0, K3), None)
((O248, E6.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



Searching with note10 constraint (20): 100%|██████████| 500/500 [00:22<00:00, 22.38it/s]


...

Searching with note10 constraint (60):   1%|          | 3/500 [00:00<00:21, 23.34it/s]

Found best architecture on note10 with latency <= 55.00 ms It achieves 84.87% predicted accuracy with 51.34 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E4.0, K5), None)
((O32, E3.0, K5), Identity)
((O32, E4.0, K3), Identity)
((O32, E4.0, K3), Identity)
((O56, E6.0, K5), None)
((O56, E4.0, K5), Identity)
((O56, E4.0, K3), Identity)
((O56, E6.0, K5), Identity)
((O104, E4.0, K7), None)
((O104, E3.0, K7), Identity)
((O104, E3.0, K3), Identity)
((O104, E6.0, K7), Identity)
((O128, E6.0, K3), None)
((O128, E6.0, K5), Identity)
((O128, E4.0, K3), Identity)
((O128, E4.0, K7), Identity)
((O248, E6.0, K5), None)
((O248, E6.0, K5), Identity)
((O248, E6.0, K5), Identity)
((O248, E3.0, K7), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



Searching with note10 constraint (60): 100%|██████████| 500/500 [00:22<00:00, 22.27it/s]

Found best architecture on note10 with latency <= 60.00 ms It achieves 84.67% predicted accuracy with 57.44 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E6.0, K3), None)
((O32, E6.0, K3), Identity)
((O32, E3.0, K5), Identity)
((O32, E4.0, K3), Identity)
((O56, E3.0, K7), None)
((O56, E6.0, K5), Identity)
((O56, E4.0, K5), Identity)
((O56, E6.0, K7), Identity)
((O104, E6.0, K7), None)
((O104, E4.0, K5), Identity)
((O104, E6.0, K7), Identity)
((O104, E6.0, K5), Identity)
((O128, E6.0, K5), None)
((O128, E6.0, K7), Identity)
((O128, E4.0, K5), Identity)
((O128, E4.0, K3), Identity)
((O248, E6.0, K5), None)
((O248, E6.0, K7), Identity)
((O248, E6.0, K7), Identity)
((O248, E3.0, K7), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



{(15,): 29.567721128463745,
 (15, 20): 57.13128590583801,
 (15, 20, 25): 89.62181806564331,
 (15, 20, 25, 30): 119.58368682861328,
 (15, 20, 25, 30, 35): 130.09224700927734,
 (15, 20, 25, 30, 35, 40, 45, 50, 55, 60): 231.71896505355835}

## Accuracy

### MobileNetV3

### ResNet50D

### ProxylessNAS

## Cost of finding a single latency constraint subnetwork

### MobileNetV3

In [24]:
times = {}
latency_constraints = (15, 20, 25, 30, 35, 40, 45, 50, 55, 60)

In [26]:
for lc in latency_constraints:
    times[lc] = run_vanilla_evolutionary_search([lc])
times

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:21<00:00, 23.68it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.48% predicted accuracy with 14.96 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E4.0, K3), Identity)
((O56, E3.0, K7), None)
((O56, E6.0, K3), Identity)
((O104, E4.0, K5), None)
((O104, E3.0, K7), Identity)
((O104, E4.0, K5), Identity)
((O128, E4.0, K5), None)
((O128, E4.0, K3), Identity)
((O128, E3.0, K3), Identity)
((O248, E4.0, K5), None)
((O248, E4.0, K3), Identity)
((O248, E4.0, K3), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



Searching with note10 constraint (25):   0%|          | 2/500 [00:00<00:25, 19.42it/s]

Found best architecture on note10 with latency <= 20.00 ms It achieves 80.59% predicted accuracy with 19.86 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E3.0, K3), None)
((O32, E4.0, K3), Identity)
((O56, E4.0, K3), None)
((O56, E4.0, K3), Identity)
((O104, E4.0, K5), None)
((O104, E4.0, K7), Identity)
((O104, E3.0, K5), Identity)
((O128, E4.0, K3), None)
((O128, E4.0, K5), Identity)
((O128, E3.0, K3), Identity)
((O248, E4.0, K7), None)
((O248, E4.0, K3), Identity)
((O248, E4.0, K7), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



Searching with note10 constraint (30):   1%|          | 3/500 [00:00<00:22, 21.79it/s]

...

Searching with note10 constraint (60):   1%|          | 3/500 [00:00<00:18, 26.48it/s]

Found best architecture on note10 with latency <= 55.00 ms It achieves 84.68% predicted accuracy with 53.43 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E4.0, K5), None)
((O32, E4.0, K5), Identity)
((O32, E3.0, K5), Identity)
((O32, E4.0, K5), Identity)
((O56, E6.0, K5), None)
((O56, E4.0, K5), Identity)
((O56, E6.0, K7), Identity)
((O56, E6.0, K3), Identity)
((O104, E6.0, K5), None)
((O104, E4.0, K7), Identity)
((O104, E4.0, K5), Identity)
((O104, E6.0, K5), Identity)
((O128, E4.0, K7), None)
((O128, E6.0, K7), Identity)
((O128, E6.0, K5), Identity)
((O128, E6.0, K5), Identity)
((O248, E6.0, K7), None)
((O248, E6.0, K5), Identity)
((O248, E6.0, K5), Identity)
((O248, E4.0, K5), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



Searching with note10 constraint (60): 100%|██████████| 500/500 [00:20<00:00, 24.50it/s]

Found best architecture on note10 with latency <= 60.00 ms It achieves 85.14% predicted accuracy with 59.89 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O40_RELU6_BN
(3x3_MBConv1_RELU6_O24_BN, None)
((O32, E4.0, K5), None)
((O32, E4.0, K5), Identity)
((O32, E4.0, K3), Identity)
((O32, E4.0, K5), Identity)
((O56, E6.0, K3), None)
((O56, E6.0, K7), Identity)
((O56, E4.0, K7), Identity)
((O56, E6.0, K3), Identity)
((O104, E4.0, K7), None)
((O104, E6.0, K7), Identity)
((O104, E6.0, K5), Identity)
((O104, E6.0, K7), Identity)
((O128, E6.0, K5), None)
((O128, E6.0, K5), Identity)
((O128, E6.0, K3), Identity)
((O128, E6.0, K3), Identity)
((O248, E6.0, K5), None)
((O248, E6.0, K7), Identity)
((O248, E6.0, K3), Identity)
((O248, E3.0, K5), Identity)
((O416, E6.0, K7), None)
1x1_Conv_O1664_RELU6_BN
1664x1000_Linear



{15: 29.994638919830322,
 20: 23.231650829315186,
 25: 22.613791942596436,
 30: 24.756764888763428,
 35: 24.37135624885559,
 40: 26.74148988723755,
 45: 23.067010164260864,
 50: 24.472725868225098,
 55: 20.9291729927063,
 60: 20.441585779190063}

In [27]:
times.values()

dict_values([29.994638919830322, 23.231650829315186, 22.613791942596436, 24.756764888763428, 24.37135624885559, 26.74148988723755, 23.067010164260864, 24.472725868225098, 20.9291729927063, 20.441585779190063])

### ResNet50D

### ProxylessNAS

247.85819578170776

# DEMO

In [26]:
latency_constraints = (15, 20, 25, 30, 35)

In [27]:
run_vanilla_evolutionary_search(latency_constraints)

Searching with note10 constraint (15): 100%|██████████| 500/500 [00:22<00:00, 22.07it/s]


Found best architecture on note10 with latency <= 15.00 ms It achieves 78.61% predicted accuracy with 14.93 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E4.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E3.0, K3), Identity)
(SE(O48, E3.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E3.0, K7), Identity)
((O96, E3.0, K5), Identity)
(SE(O136, E4.0, K3), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E4.0, K3), None)
(SE(O192, E4.0, K3), Identity)
(SE(O192, E4.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (25):   0%|          | 0/500 [00:00<?, ?it/s]

Found best architecture on note10 with latency <= 20.00 ms It achieves 80.46% predicted accuracy with 19.86 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E3.0, K3), None)
((O32, E3.0, K3), Identity)
(SE(O48, E4.0, K3), None)
(SE(O48, E4.0, K3), Identity)
(SE(O48, E4.0, K3), Identity)
((O96, E4.0, K7), None)
((O96, E4.0, K3), Identity)
(SE(O136, E3.0, K7), None)
(SE(O136, E4.0, K5), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E4.0, K3), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (30):   1%|          | 3/500 [00:00<00:21, 23.14it/s]

...

Searching with note10 constraint (60):   1%|          | 3/500 [00:00<00:22, 21.98it/s]

Found best architecture on note10 with latency <= 55.00 ms It achieves 84.93% predicted accuracy with 54.66 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E4.0, K5), None)
((O32, E6.0, K5), Identity)
((O32, E6.0, K3), Identity)
(SE(O48, E6.0, K5), None)
(SE(O48, E6.0, K5), Identity)
(SE(O48, E3.0, K7), Identity)
(SE(O48, E3.0, K5), Identity)
((O96, E4.0, K7), None)
((O96, E3.0, K7), Identity)
((O96, E6.0, K3), Identity)
((O96, E6.0, K7), Identity)
(SE(O136, E6.0, K3), None)
(SE(O136, E6.0, K5), Identity)
(SE(O136, E6.0, K3), Identity)
(SE(O136, E4.0, K3), Identity)
(SE(O192, E6.0, K5), None)
(SE(O192, E6.0, K3), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E4.0, K7), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



Searching with note10 constraint (60): 100%|██████████| 500/500 [00:24<00:00, 20.69it/s]

Found best architecture on note10 with latency <= 60.00 ms It achieves 84.65% predicted accuracy with 59.98 ms latency on note10.
Architecture of the searched sub-net:
3x3_Conv_O24_H_SWISH_BN
(3x3_MBConv1_RELU_O24_BN, Identity)
((O32, E6.0, K5), None)
((O32, E3.0, K5), Identity)
((O32, E3.0, K7), Identity)
(SE(O48, E6.0, K5), None)
(SE(O48, E6.0, K5), Identity)
(SE(O48, E4.0, K5), Identity)
(SE(O48, E6.0, K3), Identity)
((O96, E4.0, K5), None)
((O96, E6.0, K7), Identity)
((O96, E4.0, K7), Identity)
((O96, E6.0, K5), Identity)
(SE(O136, E6.0, K7), None)
(SE(O136, E6.0, K7), Identity)
(SE(O136, E6.0, K7), Identity)
(SE(O136, E6.0, K5), Identity)
(SE(O192, E6.0, K3), None)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E6.0, K5), Identity)
(SE(O192, E4.0, K5), Identity)
1x1_Conv_O1152_H_SWISH_BN
1x1_Conv_O1536_H_SWISH
1536x1000_Linear



246.21814894676208